# 1. パーセプトロンと論理回路
---
以下のような条件によって0,1を出力する関数をパーセプトロンと呼ぶ。
$$
f(x,y) = \begin{cases}
 0 & (b + x\omega_1 + y\omega_2 \leq 0)\\
  1 & (b + x\omega_1 + y\omega_2 > 0)
  \end{cases}
$$
$\omega_1,\omega_2$を$x,y$の重み、$b$をバイアスと呼ぶ。

論理回路AND,OR,NANDはパーセプトロンを用いて適切な$(b,\omega_1,\omega_2)$のもと表現することが出来る。

| (x,y) | AND | OR | NAND |
|:-----:|:-----:|:-----:|:-----:|
| (0,0) | 0 | 0 | 1 |
| (1,0) | 0 | 1 | 1 |
| (0,1) | 0 | 1 | 1 |
| (1,1) | 1 | 1 | 0 |
|(b,$\omega_1,\omega_2$) | (-1.5,1,1) | (-0.5,1,1) | (1.5,-1,-1)|

その他に重要な論理回路としてXORが上げられるが、XORは１層のパーセプトロンでは表現することが出来ない。
代わりに２層のパーセプトロンによって、
$$ XOR(x,y) = AND( NAND(x,y), OR(x,y) ) $$
と表現することが出来る。

論理回路てしてのパーセプトロンは例えばpythonを用いて以下のように書ける






In [1]:
import numpy as np
import copy

In [5]:
def AND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([1.0, 1.0])
    b = -1.5
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

In [6]:
def NAND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([-1.0, -1.0])
    b = 1.5
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

In [9]:
def OR(x1, x2):
    x = np.array([x1, x2])
    w = np.array([1.0, 1.0])
    b = -0.5
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

In [7]:
def XOR(x1, x2):
    s1 = NAND(x1, x2)
    s2 = OR(x1, x2)
    y = AND(s1, s2)
    return y

In [10]:
[AND(0,1), OR(0,1), XOR(0,1)]

[0, 1, 1]

# 2. ニューラルネットワーク
---

ここまで考えてきたパーセプトロンは、実は単純パーセプトロンと呼ばれるもので、入出力は0,1であった。
ここでは、より一般化したものとして、ニューラルネットワークを導入したい。以下のような一般化をする。
- 入力を実数にする(複素数にするとどうなるか？)
- $a=b+\vec{w}\cdot\vec{x}$が0より大きいかで0,1を出力をしていたが(すなわち関数で表現するとステップ関数$\theta(a)$を用いていたが)、別の関数系も用いることが出来る。一般的なのは、シグモイド関数($f(a)=1/(1+\exp(-a))$)と、ReLU関数($f(a) = \mathrm{max}(0,a)$)である。これらのaから出力を得る関数を活性化関数と呼ぶ
- よって出力も一般の実数となる

ここで一つ重要なのは、活性化関数は非線形な関数である必要があることである。例えば線形な関数$f(a)=pa+q$であった場合、ニューラルネットワークを多層にする意味がなくなってしまう。($f(f(a)) = p^2 a + q(p+1)$のように、１層のニューラルネットワークで表現できてしまう。)

最後の出力層については、どのような問題を考えるかによって関数を変える。
回帰問題といわれる、連続的な数値を求める問題については出力層は恒等式でそのまま与える。
分類問題といわれる、与えられた場合のいずれに分類されるかを求める問題については、ソフトマックス関数を用いるのが一般的である。ソフトマックス関数は以下の式で表される。
$$y_k = \exp[a_k]/(\sum_k \exp[a_k])$$
思想としては、微妙な分類の判定を際立たせるために指数関数の方に数値を載せて、また確率的な解釈をするために全体の和で規格化している。

以下、pythonによる実装を考える。バイアスとして各層に常に１の値を持つニューラルネットワークを配置する。また次の層に移る際の重み付けを、行列を用いて表現すれば良い。
まず活性化関数を実装する。


In [2]:
def step_fn(x):
    return np.array(x>0, dtype=np.int)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def ReLU(x):
    return np.maximum(0,x)

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)
    exp_sum = np.sum(exp_a)
    y = exp_a / exp_sum
    return y

In [15]:
def init_network():
    network = {}
    network['W1'] = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
    network['b1'] = np.array([0.1,0.2,0.3])
    network['W2'] = np.array([[0.1,0.4],[0.2,0.5],[0.3,0.6]])
    network['b2'] = np.array([0.1,0.2])
    network['W3'] = np.array([[0.1,0.3],[0.2,0.4]])
    network['b3'] = np.array([0.1,0.2])

    return network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = copy.copy(a3)

    return y

In [16]:
network = init_network()
x = np.array([1.0,0.5])
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


# 3. ニューラルネットワークの学習
----

### 3.1. 損失関数
ニューラルネットワークの予測の精度（の悪さ）を示す指標・関数を損失関数と呼ぶ。この損失関数の勾配を用いて、パラメータの更新を行い学習を推進していく。

良く用いられるものは、2乗和誤差と交差エントロピー誤差である。それぞれ以下の式で表される。
$$
Err = \begin{cases}
\frac{1}{2}\sum_k(y_k-t_k)^2 & (\mathrm{mean \ square \ error})\\
-\sum_k t_k \mathrm{log}y_k & (\mathrm{cross \ entropy \ error})
\end{cases}
$$
どちらも予測結果$y_k$が教師結果$t_k$に等しければ0になるようになっている。（正直もっと良い損失関数ありそう）

以下pythonによる実装である。実際にはいくつかのデータにおける損失の平均をとってその精度を測定することになるので、そのように実装している。


In [17]:
def MSE(y,t):
    batch_size = y.shape[0]
    return 0.5 * np.sum((y-t)**2) / batch_size

def CEE(y,t):
    delta = 0.00001
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + delta)) / batch_size

def Grad(f, x):
    dx = 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        temp = x[idx]
        x[idx] = temp + dx
        y1 = f(x)
        x[idx] = temp - dx
        y2 = f(x)
        grad[idx] = (y1 -y2) / (2 * dx)
        x[idx] = temp
    
    return grad


In [18]:
#2層ニューラルネットワークのクラス
import sys, os
sys.path.append(os.pardir)

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    def loss(self, x, t):
        y = self.predict(x)
        return CEE(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        ya = np.argmax(y, axis=1)
        ta = np.argmax(t, axis=1)

        acc = np.sum(y == t) / float(x.shape[0])
        return acc

    def Num_Grad(self, x, t):
        def loss_W(W):
            return self.loss(x,t)
        
        grads = {}
        grads['W1'] = Grad(loss_W, self.params['W1'])
        grads['b1'] = Grad(loss_W, self.params['b1'])
        grads['W2'] = Grad(loss_W, self.params['W2'])
        grads['b2'] = Grad(loss_W, self.params['b2'])

        return grads

    


In [4]:
import tensorflow as tf
from keras.utils.np_utils import to_categorical

mnist = tf.keras.datasets.mnist
mnist

def load_mnist(normalize = False, flatten = False, one_hot_label = False):
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  if (normalize):
    x_train = x_train / 255
    x_test = x_test / 255

  if (flatten):
    x_train = x_train.reshape(len(x_train), -1)
    x_test = x_test.reshape(len(x_test), -1)

  if (one_hot_label):
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)


  return (x_train,y_train), (x_test,y_test)

In [16]:

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True,flatten=True, one_hot_label=True)
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

train_loss_list = []
Ite = []

iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
print(network.params['W1'].shape)
print(network.params['b1'].shape)
print(network.params['W2'].shape)
print(network.params['b2'].shape)
print(network.params['W1'].size)


for i in range(iters_num):

    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.Num_Grad(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    Ite.append(i)

import matplotlib.pyplot as plt

plt.plot(Ite, train_loss_list, label="loss")
plt.xlabel("Iterartion")
plt.ylabel("Loss")
plt.legend()
plt.show()



(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)
(784, 50)
(50,)
(50, 10)
(10,)
39200


IndexError: index 784 is out of bounds for axis 0 with size 784